Recorded ping tests: 1 per minute, single ping, to 8.8.8.8 (google dns)

Data quality notes: At one point in time we had a timezone jump because the timezone was corrected on the measuring device. Timezone was recorded, but I chose to keep processing simple and ignore timezones as much as possible

In [1]:
# Input file: contains output of 'date' and 'ping'
!head -n 20 googledns.log

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 17.268/17.268/17.268/0.000 ms
Sat 01 Aug 2020 12:00:01 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=51.9 ms

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 51.918/51.918/51.918/0.000 ms
Sat 01 Aug 2020 12:01:01 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=19.2 ms

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 19.151/19.151/19.151/0.000 ms
Sat 01 Aug 2020 12:02:02 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=20.1 ms


In [18]:
import re

In [23]:
lines = open('googledns.log').readlines()
lines = [l.strip() for l in lines]
datepattern = re.compile('.*2020.*')
dates = list(filter(lambda line: datepattern.match(line), lines))
print("First and last timestamp:")
print(dates[0])
print(dates[-1])

# here you can see that TZ change mentioned at the top

First and last timestamp:
Sat 01 Aug 2020 12:00:01 AM BST
Sun 09 Aug 2020 08:59:01 PM CEST


In [24]:
resultpattern = re.compile('1 packets transmitted')
results = list(filter(lambda line: resultpattern.match(line), lines))
print("Expected # ping tests:", 60*24*8 + 60*21) # about 8 full days + 21 hours on the last date
print("Actual # ping tests:", len(results))

# looks good - again, off by 60 because of the TZ change

Expected # ping tests: 12780
Actual # ping tests: 12719


In [34]:
# Now let's check successful & failed pings
success_count = len(list(filter(lambda result: '1 received' in result,results)))
fail_count = len(list(filter(lambda result: '0 received' in result,results)))
print("Successful:", success_count)
print("Failures  :", fail_count)
print("Total     :", (success_count + fail_count))

Successful: 11060
Failures  : 1659
Total     : 12719


In [38]:
# How many hours of failure is that?
print("Estimated hour of failure:", (fail_count / 60))

Estimated hour of failure: 27.65
